## First let's do some symbolic derivation

In [ ]:
from IPython.display import Math,display
import sympy

In [ ]:
xs=x0,x1,x2,x3=sympy.symbols('x:4')
ys=y0,y1,y2,y3=sympy.symbols('y:4')
xy=sympy.Matrix([xs,ys]).T
display(xy)

In [ ]:
# corrdinate of parent domain
xi,eta=sympy.symbols('xi eta')
# shape functions
N=sympy.Rational(1,4)*sympy.Matrix([(1-xi)*(1-eta),(1+xi)*(1-eta),(1+xi)*(1+eta),(1-xi)*(1+eta)]).T
display(N)
# partial(N1,N2,N3,N4)/partial(xi,eta)
GN=sympy.Matrix([[sympy.diff(i,xi) for i in N],[sympy.diff(i,eta) for i in N]])
display(GN)

In [ ]:
# J is partial(x,y)/partial(xi,eta)
J=sympy.simplify(GN*xy)
display(J)
Jdet=sympy.simplify(J.det())
Jinv=sympy.simplify(J**(-1))
# partial(N1,N2,N3,N4)/partial(x,y)
NablaN=sympy.simplify(Jinv*GN)

In [ ]:
Ksym=[]
for i in range(4):
    Ksym.append([])
    for j in range(0,i+1):
        integrand=sympy.simplify(Jdet*(NablaN[:,i].T*NablaN[:,j])[0])
        Ksym[-1].append(integrand)

## Then I read a mesh exported from blender

In [ ]:
#%matplotlib notebook
import meshio,numpy
meshio.__version__
import matplotlib.pyplot as plt

In [ ]:
sample=meshio.read("square5x5.ply")
#sample=meshio.read("square20x20.ply")
print(sample)

In [ ]:
points=sample.__dict__['points'][:,0:2]
cells=sample.cells_dict['quad']
Npts=len(points)
print(points)
print(cells)

In [ ]:
textdx=0.2*(2/numpy.sqrt(Npts))
plt.scatter(points[:,0],points[:,1],s=4)
for i,p in enumerate(points):
    plt.text(p[0]+textdx,p[1],str(i),fontsize=12)
plt.show()

In [ ]:
# define boundary conditions
def bdfun(p):
    return p[0]*1+p[1]*2

counter=numpy.zeros(Npts,dtype=numpy.uint)
for c in cells:
    for i in c:
        counter[i]+=1
        
bd1st={};bd2nd={}
for i in range(Npts):
    if counter[i]==4:
        continue
    if numpy.isclose(points[i][1],-1) or numpy.isclose(points[i][1],1) or\
       numpy.isclose(points[i][0],-1) or numpy.isclose(points[i][0],1):
        bd1st[i]=bdfun(points[i])
    else:
        bd2nd[i]=None
print("bd1st (len=%d):\n"%(len(bd1st)),bd1st)
print("")
print("bd2nd (len=%d):\n"%(len(bd2nd)),bd2nd)

In [ ]:
counter=0
g2i={}
for i in range(Npts):
    if i not in bd1st:
        g2i[i]=counter
        counter+=1
Nipts=len(g2i)
print(g2i)

## Then begins the real FEM

In [ ]:
from tqdm import tqdm
import numpy,scipy.integrate,scipy.sparse.linalg

In [ ]:
def gen_K_b(kappa=1.0):
    Kval=[];Kij=[]
    b=numpy.zeros(Nipts)
    for c in tqdm(cells):
        gen_K_cell(c,Kval,Kij,b,kappa)
    row=[i[0] for i in Kij]
    col=[i[1] for i in Kij]
    K=scipy.sparse.coo_matrix((Kval,(row,col)),shape=(Nipts,Nipts))
    return K.tocsr(),b

def gen_K_cell(cell,Kval,Kij,b,kappa):
    xyval=[(xs[i],points[p][0]) for i,p in enumerate(cell)]+[(ys[i],points[p][1]) for i,p in enumerate(cell)]
    for i in range(4):
        if cell[i] not in bd1st:
            f=sympy.lambdify([xi,eta],Ksym[i][i].subs(xyval))
            k=kappa*scipy.integrate.dblquad(f,-1,1,-1,1)[0]
            indi=g2i[cell[i]]
            Kval.append(k)
            Kij.append((indi,indi))

        for j in range(i):
            if cell[i] in bd1st and cell[j] in bd1st:
                continue
            
            f=sympy.lambdify([xi,eta],Ksym[i][j].subs(xyval))
            k=kappa*scipy.integrate.dblquad(f,-1,1,-1,1)[0]
            
            
            if cell[i] in bd1st:
                #print("detect boundary: %d->%d"%(cell[j],cell[i]))
                indj=g2i[cell[j]]
                b[indj]-=bd1st[cell[i]]*k
                del indj
            elif cell[j] in bd1st:
                #print("detect boundary: %d->%d"%(cell[i],cell[j]))
                b[indi]-=bd1st[cell[j]]*k
            else:
                indj=g2i[cell[j]]
                Kval.append(k);Kval.append(k)
                Kij.append((indi,indj))
                Kij.append((indj,indi))
        try:
            del indi
        except:
            pass
        try:
            del indj
        except:
            pass

K,b=gen_K_b()

In [ ]:
print("K:\n",K)
print("b:\n",b)

In [ ]:
x=scipy.sparse.linalg.spsolve(K,b)
#print("x:",x)
gx=numpy.zeros(Npts)
for i in range(Npts):
    if i in bd1st:
        gx[i]=bd1st[i]
    else:
        gx[i]=x[g2i[i]]

In [ ]:
def plot3d(gx):
    fig=plt.figure()
    ax=fig.add_subplot(projection='3d')
    drawn=set()
    for c in cells:
        for i,j in ((0,1),(1,2),(2,3),(3,0)):
            if (c[i],c[j]) in drawn:
                continue
            xs=[points[c[i]][0],points[c[j]][0]]
            ys=[points[c[i]][1],points[c[j]][1]]
            zs=[gx[c[i]],gx[c[j]]]
            ax.plot(xs,ys,zs,'b')
            drawn.add((c[i],c[j]))
    ax.azim=-10
    plt.show()
    
plot3d(gx)